In [1]:
import pandas as pd
import numpy as np
from MDAnalysis import Universe
from MDAnalysis.analysis.distances import distance_array
from calc_df import *
from utils import *

/opt/anaconda3/envs/ML_learn/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_pairs = [
    ('../../resub_traj/n17_k10_II.red.tpr', '../../resub_traj/n17_k10_II.red.xtc'),
    ('../../resub_traj/n17_k10_III.red.tpr', '../../resub_traj/n17_k10_III.red.xtc'),
    ('../../resub_traj/n17_k10_IV.red.tpr', '../../resub_traj/n17_k10_IV.red.xtc')
]

# Create Skeleton DataFrame

Each row of the DataFrame contains the following columns:

- **`global_pep_ID`**  
  Peptide count across replicas (ranging grom 0 to 35*3)

- **`local_pep_ID`**  
  Peptide index within a single replica (ranges from 0 to 35).

- **`frame`**  
  Frame number (ranges from 0 to 5000).

- **`trajectory`**  
  Identifier for the trajectory the peptide-frame pair comes from.

- **`target_status`**  
  Categorical label (0–3) indicating peptide binding type:

  | Value | Description                    |
  |-------|--------------------------------|
  | 0     | Directly bound to **Mc**       |
  | 1     | Directly bound to **Mp**       |
  | 2     | Indirectly bound to **Mc**     |
  | 3     | Indirectly bound to **Mp**     |


In [7]:
# Function calls to generate the initial template df
dfs = []

for i, (tpr, xtc) in enumerate(file_pairs):
    df_traj = build_init_df(tpr, xtc, traj_index=i, start=0, stop=5000)
    dfs.append(df_traj)

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv("skeleton_df.csv", index=False)

In [8]:
df_template = pd.read_csv('skeleton_df.csv')
print_df_status(df_template)

Current version of the df has the following columns
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory',
       'target_status'],
      dtype='object')


## Peptide-peptide heavy atom contacts

Calculates # of contacts (within 7 Å)  between a selected peptide and all others across specified frames and trajectories, returning a DataFrame of contact counts.



In [12]:
# Run the peptide-peptide contact analysis
contact_df = pep_pep_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "pp_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)

save_new_df(df_template)
print_df_status(df_template)

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts'],
      dtype='object')
Saving to current_features.csv
Current version of the df has the following columns
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts'],
      dtype='object')


## Peptide-lipid interactions


Calculates hydrogen-like contacts (within 7 Å) between a selected peptide and all lipids (PO4)  across specified frames and trajectories, returning a DataFrame of contact counts.


In [16]:
# Run the peptide-peptide contact analysis
contact_df = pep_lip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "pl_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)

save_new_df(df_template)
print_df_status(df_template)

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts'],
      dtype='object')
Saving to current_features.csv
Current version of the df has the following columns
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts'],
      dtype='object')


## Dipole peptide-peptide interactions

In [19]:
# Load latest version of df and run the peptide-peptide contact analysis
df_template = pd.read_csv('current_features.csv')
contact_df = pep_dip_dip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "dip_pp_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts'],
      dtype='object')
Saving to current_features.csv


## Dipole peptide-lipid interactions


Counts close contacts (<3.5 Å) between dipole-heavy atoms of selected peptides and lipid atoms (POPC whole) across specified frames and trajectories.

In [27]:
# Load latest version of df and run the peptide-peptide contact analysis
df_template = pd.read_csv('current_features.csv')
contact_df = dip_pep_lip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "dip_lp_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts', 'dip_lp_contacts_x',
       'dip_lp_contacts_y'],
      dtype='object')
Saving to current_features.csv


In [32]:
df_template.drop(columns='dip_lp_contacts_x',inplace=True)
df_template.rename(columns={'dip_lp_contacts_y': 'dip_lp_contacts'}, inplace=True)

In [33]:
df_template.head(10)

,global_pep_ID,local_pep_ID,frame,trajectory,target_status,pp_contacts,pl_contacts,dip_pp_contacts,dip_lp_contacts
0,10,10,83,0,3,0,0,0,0
1,22,22,83,0,1,0,10,0,0
2,34,34,83,0,3,0,0,0,0
3,10,10,85,0,3,0,0,0,0
4,22,22,85,0,1,0,8,0,1
5,34,34,85,0,3,0,0,0,0
6,10,10,86,0,3,0,0,0,0
7,22,22,86,0,1,0,7,0,3
8,34,34,86,0,3,0,0,0,0
9,10,10,87,0,3,0,0,0,0


In [34]:
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts', 'dip_lp_contacts'],
      dtype='object')
Saving to current_features.csv


In [3]:
# Load latest version of df and run the peptide-peptide contact analysis
df_template = pd.read_csv('current_features.csv')
contact_df = hyd_pep_lip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "hyd_lp_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts', 'dip_lp_contacts',
       'hyd_lp_contacts'],
      dtype='object')
Saving to current_features.csv


In [3]:
print_df_status(df_template)

NameError: name 'df_template' is not defined

## Positive peptide SC with negative PO4


In [5]:
df_template = pd.read_csv('current_features.csv')
contact_df = Ppep_Nlip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "Ppep_Nlip_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts', 'dip_lp_contacts',
       'hyd_lp_contacts', 'Ppep_Nlip_contacts'],
      dtype='object')
Saving to current_features.csv


## Negative peptide SC with positive NC3


In [6]:
contact_df = Npep_Plip_contacts(df_template, file_pairs)

# Merge the new contact data back into the base DataFrame
df_template = pd.merge(
    df_template,
    contact_df[["trajectory", "frame", "local_pep_ID", "Npep_Plip_contacts"]],
    on=["trajectory", "frame", "local_pep_ID"],
    how="left"
)
save_new_df(df_template)
df_template = pd.read_csv('current_features.csv')

Saving current version of the df. 
 The following columns are present: 
Index(['global_pep_ID', 'local_pep_ID', 'frame', 'trajectory', 'target_status',
       'pp_contacts', 'pl_contacts', 'dip_pp_contacts', 'dip_lp_contacts',
       'hyd_lp_contacts', 'Ppep_Nlip_contacts', 'Npep_Plip_contacts'],
      dtype='object')
Saving to current_features.csv
